In [1]:
#https://data.worldbank.org/topic/19

In [2]:
def Shape(df): return df.count(), len(df.columns)
def Display(df, row=5): return df.limit(row).toPandas().head(row)

In [3]:
import pyspark

import pandas as pd
import pyspark.sql.functions as F

from pyspark.sql import SparkSession
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
from pyspark.sql.window import Window

In [4]:
#InteractiveShell.ast_node_interactivity = all
spark = SparkSession.builder.appName('Practise').getOrCreate()
pd.set_option('max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
df = spark.read.csv('API_19_DS2_en_csv_v2_5873088.csv', header=True, inferSchema=True, sep=';')

In [6]:
Shape(df)

(20216, 67)

In [7]:
year_cols = [str(i) for i in range(1960,2023)]
for col in year_cols:
    df = df.withColumn(col, F.when(F.col(col).isNull(), "NULL").otherwise(F.col(col)))
    df = df.withColumn(col, F.array(F.lit(col), col))

In [8]:
df = df.withColumn("year_cols", F.array(year_cols)) \
       .drop(*year_cols) \
       .drop("Country Code", "Indicator Code")

In [9]:
temp = df.groupBy("Country Name") \
         .agg(F.collect_list(F.array(F.array(F.array(F.col("Indicator Name"))), F.col("year_cols"))).alias("temp") ) \
         .orderBy("Country Name")

In [10]:
temp2 = temp.select("Country Name", F.explode("temp").alias("temp")) \
            .withColumn("Type", F.col("temp").getItem(0).getItem(0).getItem(0)) \
            .withColumn("col2", F.col("temp").getItem(1)) \
            .select("Country Name", "Type", F.explode("col2").alias("final")) \
            .withColumn("Year", F.col("final").getItem(0)) \
            .withColumn("Value", F.col("final").getItem(1)) \
            .drop("final") \
            .replace('NULL', None)

In [11]:
final_df = temp2.groupBy("Country Name", "Year").pivot("Type").agg(F.first("Value")).orderBy("Country Name", "Year")

In [12]:
Shape(final_df)

(16758, 78)

In [13]:
Display(final_df, 10)

,Country Name,Year,Access to electricity (% of population),Agricultural irrigated land (% of total agricultural land),Agricultural land (% of land area),Agricultural land (sq. km),Agriculture forestry and fishing value added (% of GDP),Annual freshwater withdrawals total (% of internal resources),Annual freshwater withdrawals total (billion cubic meters),Arable land (% of land area),Average precipitation in depth (mm per year),CO2 emissions (kg per 2015 US$ of GDP),CO2 emissions (kg per 2017 PPP $ of GDP),CO2 emissions (kg per PPP $ of GDP),CO2 emissions (kt),CO2 emissions (metric tons per capita),CO2 emissions from gaseous fuel consumption (% of total),CO2 emissions from gaseous fuel consumption (kt),CO2 emissions from liquid fuel consumption (% of total),CO2 emissions from liquid fuel consumption (kt),CO2 emissions from solid fuel consumption (% of total),CO2 emissions from solid fuel consumption (kt),CO2 intensity (kg per kg of oil equivalent energy use),CPIA public sector management and institutions cluster average (1=low to 6=high),Cereal yield (kg per hectare),Community health workers (per 1000 people),Disaster risk reduction progress score (1-5 scale,Droughts floods extreme temperatures (% of population average 1990-2009),Ease of doing business rank (1=most business-friendly regulations),Electric power consumption (kWh per capita),Electricity production from coal sources (% of total),Electricity production from hydroelectric sources (% of total),Electricity production from natural gas sources (% of total),Electricity production from nuclear sources (% of total),Electricity production from oil sources (% of total),Electricity production from renewable sources excluding hydroelectric (% of total),Electricity production from renewable sources excluding hydroelectric (kWh),Energy use (kg of oil equivalent per capita),Energy use (kg of oil equivalent) per $1000 GDP (constant 2017 PPP),Foreign direct investment net inflows (% of GDP),Forest area (% of land area),Forest area (sq. km),GHG net emissions/removals by LUCF (Mt of CO2 equivalent),HFC gas emissions (thousand metric tons of CO2 equivalent),Land area where elevation is below 5 meters (% of total land area),Marine protected areas (% of territorial waters),Methane emissions (% change from 1990),Methane emissions (kt of CO2 equivalent),Mortality rate under-5 (per 1000 live births),Nitrous oxide emissions (% change from 1990),Nitrous oxide emissions (thousand metric tons of CO2 equivalent),Other greenhouse gas emissions (% change from 1990),Other greenhouse gas emissions HFC PFC and SF6 (thousand metric tons of CO2 equivalent),PFC gas emissions (thousand metric tons of CO2 equivalent),Population growth (annual %),Population in urban agglomerations of more than 1 million (% of total population),Population living in areas where elevation is below 5 meters (% of total population),Population total,Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population),Prevalence of underweight weight for age (% of children under 5),Primary completion rate total (% of relevant age group),Renewable electricity output (% of total electricity output),Renewable energy consumption (% of total final energy consumption),Rural land area where elevation is below 5 meters (% of total land area),Rural land area where elevation is below 5 meters (sq. km),Rural population living in areas where elevation is below 5 meters (% of total population),SF6 gas emissions (thousand metric tons of CO2 equivalent),School enrollment primary and secondary (gross) gender parity index (GPI),Terrestrial and marine protected areas (% of total territorial area),Terrestrial protected areas (% of total land area),Total greenhouse gas emissions (% change from 1990),Total greenhouse gas emissions (kt of CO2 equivalent),Urban land area where elevation is below 5 meters (% of total land area),Urban land area where elevation is below 5 meters (sq. km),Urban population,Urban population (% of total population),Urban popul

In [14]:
final_df.toPandas().to_csv('climate_dataset.csv') 